# ML/AI Engineering Foundations: Final Portfolio
The following Jupyter notebook was created as a part of the final project for the Codecademy course ["ML/AI Engineering Foundations"](https://www.codecademy.com/learn/paths/machine-learning-ai-engineering-foundations). 

## 